Reference: [Build a Retrieval Augmented Generation (RAG) App](https://python.langchain.com/v0.2/docs/tutorials/rag/)

# Build HF LLM

In [9]:
import torch
import pandas as pd
from langchain import hub
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFacePipeline
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextStreamer,
    pipeline,
)

# device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

model_name = "meta-llama/Meta-Llama-3.1-70B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    cache_dir="./llm_model/", trust_remote_code=True
)
streamer = TextStreamer(tokenizer=tokenizer, skip_prompt=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir="./llm_model/",
    
    device_map='auto',
    low_cpu_mem_usage=True,  # try to limit RAM
    
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        load_in_8bit_fp32_cpu_offload=True
    ), 
 # load model in low precision to save memory
    # attn_implementation="flash_attention_2",
)

pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    max_new_tokens=2048,
    streamer=streamer,
)

hfPipeline = HuggingFacePipeline(pipeline=pipe)


Unused kwargs: ['load_in_8bit_fp32_cpu_offload']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

## Build vector database

https://how.wtf/how-to-use-chroma-db-step-by-step-guide.html

In [11]:
import os
from langchain import hub
from langchain_community.document_loaders import UnstructuredMarkdownLoader, PyPDFLoader
#from langchain_community.vectorstores import Qdrant

from langchain_qdrant import QdrantVectorStore
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embedding_cache_path = "embedding_cache"
db_collection_names = ["CNS16190-zh_TW_only_provision", "ts_103701v010101p_without_content"]
document_root_path = "docs"
documents = ["CNS16190-zh_TW_only_provision.pdf", "ts_103701v010101p_without_content.pdf"]
# 創建一個空列表來保存所有分割後的文檔
all_splits = []

# 加載和分割每個文檔
for idx, document in enumerate(documents):
    if document.startswith("CNS16190"):
        collection_name = "CNS16190"
    elif document.startswith("ts_103701"):
        collection_name = "TS103701"

    if os.path.isdir(os.path.join(embedding_cache_path, collection_name)):
        # database already exists, load it
        print("Loading existing database")
        if collection_name == "CNS16190":
            vectorstore_CNS16190 = QdrantVectorStore.from_existing_collection(
                embedding=SentenceTransformerEmbeddings(model_name="lier007/xiaobu-embedding-v2"),
                path=embedding_cache_path + '/' + collection_name,
                collection_name=collection_name,
            )
            retriever_CNS16190 = vectorstore_CNS16190.as_retriever()
        elif collection_name == "TS103701":
            vectorstore_TS103701 = Qdrant.from_existing_collection(
                embedding=SentenceTransformerEmbeddings(model_name="lier007/xiaobu-embedding-v2"),
                path=embedding_cache_path + '/' + collection_name,
                collection_name=collection_name,
            )
            retriever_TS103701 = vectorstore_TS103701.as_retriever()


    else:
        # database does not exist, create it
        print("Creating new database")
        print(f"Processing document: {document}")

        if document.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(document_root_path, document))
        elif document.endswith(".md"):
            loader = UnstructuredMarkdownLoader(os.path.join(document_root_path, document), mode="elements")
        else:
            continue
        doc = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(doc)
        # 將所有分割後的文檔添加到同一個 Qdrant 集合中
        if collection_name == "CNS16190":
            vectorstore_CNS16190 = Qdrant.from_documents(
                splits,
                embedding=SentenceTransformerEmbeddings(model_name="lier007/xiaobu-embedding-v2"),
                path=embedding_cache_path + '/' + collection_name,
                collection_name=collection_name,
            )
            # 設置檢索器
            retriever_CNS16190 = vectorstore_CNS16190.as_retriever()
        elif collection_name == "TS103701":
            vectorstore_TS103701 = Qdrant.from_documents(
                splits,
                embedding=SentenceTransformerEmbeddings(model_name="lier007/xiaobu-embedding-v2"),
                path=embedding_cache_path + '/' + collection_name,
                collection_name=collection_name,
            )
            # 設置檢索器
            retriever_TS103701 = vectorstore_TS103701.as_retriever()

print("All documents have been processed and added to the Qdrant collection.")

Loading existing database
Loading existing database
All documents have been processed and added to the Qdrant collection.


In [14]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

prompt.messages[
    0
].prompt.template = "你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：{context} \n問題：{question} \n答案："

prompt

/home/m11207524/.local/lib/python3.10/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：{context} \n問題：{question} \n答案：'))])

## Build rag chain

In [17]:
from langchain_core.runnables import RunnablePassthrough

rag_chain_CNS16190 = (
    {"context": retriever_CNS16190, "question": RunnablePassthrough()} | prompt | hfPipeline
)


rag_chain_TS103701 = (
    {"context": retriever_CNS16190, "question": RunnablePassthrough()} | prompt | hfPipeline
)

In [11]:
questions = [
    "「若使用預先安裝之每裝置唯一通行碼，則應以機制產生此等通行碼，以降低對某類別或型式裝置的自動化攻擊之風險」符合哪一項控制措施？",  # 控制措施5.1-2
    "「對已揭露之脆弱性宜以及時方式採取行動」符合哪一項控制措施？",  # 控制措施5.2-2
    "「於所定義支援期間內，對其銷售及生產持續監視及矯正安全脆弱性。」符合哪一項控制措施？",  # 控制措施5.2-3（節錄）
    "「軟體以最小必要特殊權限運行」符合哪一項控制措施？",  # 控制措施5.6-7（節錄）
]

In [13]:
rag_chain_CNS16190.batch(questions)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


控制措施5.4-4。<|eot_id|>


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


控制措施5.6-2<|eot_id|>


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


控制措施5.3-14<|eot_id|>


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


控制措施5.6-7：軟體宜以最小必要特殊權限運行，同時考量安全及功能性。<|eot_id|>


["Human: 你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：[Document(page_content='控制措施5.6-\\n2：於初始化狀態下，裝置之網路介面應最小化未經鑑別的安全相關資訊揭露。\\n作為初始化過程之一部分，可經由網路介面暴露與安全相關的資訊。當裝置於建立連接共享與安\\n全相關之資訊時，攻擊者可能用以識別脆弱裝置。\\n例 \\n3.當於整個IP位址空間中查找(finding)脆弱裝置時，與安全相關之資訊可能為關於裝置組態、內核(\\nkernel)版本或軟體版本的資訊。\\n控制措施5.6-3：裝置硬體不宜將實體介面非必要暴露於攻擊。\\n攻擊者可能使用實體介面破壞裝置上之韌體或記憶體。〝非必要〞係指製造者對開啟用於使用者\\n功能性或除錯目的之介面的利益之評量。\\n例 4.旨在僅用於為裝置供電之micro-USB埠，於實體上組態設定為亦不容許執行命令或除錯。\\n控制措施5.6-4：若除錯介面係可實體存取，則應於軟體中停用。\\n例 \\n5.透過裝置上之啟動載入器軟體，停用UART串列介面。由於此停用，將無登入提示及互動式選單\\n。\\n控制措施5.6-\\n5：對裝置之預期用途或運作，製造者宜僅啟用其所使用或所要求的軟體服務。\\n例 6.對預期用途，製造者不提供裝置未要求之任何背景過程、內核延伸、命令、程式或工具。\\n控制措施5.6-6：程式碼宜最小化服務/裝置運作所必要之功能性。\\n例 7.移除〝失效〞或未使用之程式碼，其不視為良性程式碼。\\n控制措施5.6-7：軟體宜以最小必要特殊權限運行，同時考量安全及功能性。\\n例 \\n8.以〝根〞特殊權限運行最少常駐程式/過程。特別是使用網路介面之過程要求非特殊權限使用者\\n而要求非〝根〞使用者。\\n例9. \\n對各組件或服務，於納入多使用者作業系統(例：Linux®)之裝置上運行的應用程式，使用不同之使\\n用者。', metadata={'source': 'docs\\\\CNS16190-zh_TW_only_provision.pdf', 'page': 12, '_id': '8a5317cc67b049869a464ccc9a24ae26', '_collection_name': 'CNS16190'})

## Ask llm

In [18]:
df = pd.read_csv("docs/test_data_from_GoogleSheet.csv")

df

,provision,status,support,detail,Test Scenario,answer,reason,result
0,5.1-1,MC,NaN,如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的...,問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置...,使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或...,NaN,NaN
1,5.1-3,M,NaN,用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。,與裝置所使用的身份驗證方法無關，身份驗證資料在使用者和裝置之間傳輸時應進行加密。注意：傳輸通...,本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 透過 HTTPs TL...,NaN,NaN
2,5.2-1,M,NaN,製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊;以及關於以下方面的時程表的資...,漏洞揭露政策明確規定了安全研究人員和其他人報告問題的流程。 此類政策可以根據需要進行更新，以...,通過D-Link 網站（https://support.dlink.com/ReportVu...,NaN,NaN
3,5.3-13,M,NaN,製造商應以對使用者易於理解、透明的方式公開定義的支援期限。,消費者在購買設備時，希望能明確說明該期限的軟體更新支援。問題：1. 客戶如何獲知定義的支援期...,產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。,NaN,NaN
4,5.3-16,M,NaN,消費者物聯網設備的型號名稱應通過設備上的標籤或通過實體介面清晰可識別。,問題：1. 使用者如何驗證設備型號名稱（透過設備上的標籤或透過實體介面）？,此產品的型號標籤上有相關的生產編號資訊。,NaN,NaN
5,5.4-1,M,NaN,持久存儲中的敏感安全參數應由設備安全的存儲。,安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉...,所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。,NaN,NaN
6,5.4-3,M,NaN,關鍵安全參數不得硬編碼於使用設備軟體原始碼中。,設備和應用程式的逆向工程可以輕鬆發現憑證，例如軟體中硬編碼的使用者名稱和密碼。注意：此類憑證...,D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded c...,NaN,NaN
7,5.4-4,M,NaN,用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個...,為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果...,D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\r\n...,NaN,NaN
8,5.5-1,M,NaN,消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。,安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,NaN,NaN
9,5.5-5,M,NaN,設備功能只有在認證存取之後允許透過網路介面進行配置安全相關修改。例外情況是設備所依賴的網路服...,注意：例外的協定包括 ARP、DHCP、DNS、ICMP 和 NTP。範例：與安全性相關的變...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,NaN,NaN


### One row testing

In [21]:
row = df.iloc[8]
# query = f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
query = f"控制措施{row['provision']}，detail:{row['detail']}"
print(
    f"控制措施 {row['provision']}\n\n【detail】\n{row['detail']}\n\n【question】\n{row['Test Scenario']}\n\n【answer】\n{row['answer']}"
)

print(f"\n【Test scenario】\n{row['Test Scenario']}")


控制措施 5.5-1

【detail】
消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。

【question】
安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，很難提供有關密碼學或其他安全措施的規範性建議，而不會面臨此類建議很快就會過時的風險。問題：1. 消費者物聯網設備使用什麼最佳實踐加密技術來安全通訊？

【answer】
D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密方式進行，並採用最高等級的Cipher 符合FIPS 14--3 的安全強度模組進行，並且會隨時注意最先安全通報，檢查是否現行加密模組是否被發現漏洞，需要時即時更新。

【Test scenario】
安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，很難提供有關密碼學或其他安全措施的規範性建議，而不會面臨此類建議很快就會過時的風險。問題：1. 消費者物聯網設備使用什麼最佳實踐加密技術來安全通訊？


In [22]:
# rag_chain.invoke(f"{query}\n\n根據控制措施和detail在ETSI TS 103 701中找出與控制措施最相關Test group，\
#                  並生成測試流程來供測試者驗證answer 是否真的符合控制措施detail要求。")
# rag_chain_CNS16190.invoke(f"{query}\n\n根據控制措施和detail在CNS16190中找出這個控制措施到下一個控制措施之間的詳細內容。\
#                  輸出格式為: 控制措施{row['provision']}: 控制措施的詳細內容")
result_CNS16190 = rag_chain_CNS16190.invoke(f"{query}\n\n根據控制措施{row['provision']}和{row['detail']}\
                                            在CNS16190中找出這個控制措施到下一個控制措施之間的所有內容。\
                                            輸出格式為: 控制措施{row['provision']}: 控制措施的內容")

print(result_CNS16190)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KeyboardInterrupt: 

In [17]:
# rag_chain_TS103701.invoke(f"{query}\n\n根據控制措施和detail在根據控制措施和detail在ETSI TS 103 701中\
#                           找出The test group addresses the provision {row['provision']}.內容。\
#                  HINT: 控制措施英文是provision")
result_TS103701 = rag_chain_TS103701.invoke(f"{query}\n\n在ETSI TS 103 701中找出{row['provision']}Test group{row['provision']}到下一個Test group之間的內容，\
                          HINT: 控制措施就是provision。\
                          輸出格式為: Test group{row['provision']}內容: ")
# rag_chain_TS103701.invoke(f"在ETSI TS 103 701中\
#                           找出The test group addresses the provision 5.3-14.內容。")
print(result_TS103701)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


控制措施5.5-1：消費者IoT裝置應使用最佳實務密碼學安全通訊。安全控制措施之適切性及最佳實務密碼學的使用，取決於許多因素，包括使用全景。隨著安全不斷演進，很難提供關於密碼學或其他安全措施之規定性建議，而無此種建議很快過時的風險。<|eot_id|>
Human: 你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。
背景資料：[Document(page_content='控制措施5.1-\n3：用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之\n最佳實務密碼學。\n控制措施5.1-\n4：當使用者可對裝置鑑別時，裝置應向使用者或管理者提供簡單機制，以變更所\n使用之鑑別值。\n例3.對生物特徵鑑別值，裝置製造者容許透過對新生物特徵之重新訓練變更鑑別值。\n例4.家庭中之父母於裝置上為其子女建立帳戶，選擇並管理子女所使用的PIN或通行碼。父母係裝\n置管理者，可限制子女變更PIN或通行碼。\n例5.為讓使用者易於變更通行碼，製造者以要求最少步驟之方式，設計通行碼變更過程。製造者\n於使用者手冊及教學影片中解釋該過程。\n用以鑑別使用者之鑑別機制，無論其為指紋、通行碼或其他符記，其值需可更改。當此機制係裝\n置正常使用流程之一部分時，將較容易。\n控制措施5.1-\n5：當裝置係非受限制時，其應具可用之機制，使得經由網路介面暴力攻擊鑑別機\n制為不切實際。\n例6.裝置於一定時間區間內限制鑑別嘗試之次數，亦使用增加嘗試間之時間間隔。\n例7.客戶端應用程式能於鑑別嘗試不成功之限定次數後，鎖住帳戶或延遲後續鑑別嘗試。\n此控制措施因應執行〝信符填充(credential \nstuffing)〞或窮舉整個金鑰空間之攻擊。重要的是，消費者IoT裝置可偵測出此等型式之攻擊並加\n以防禦，同時防範相關的〝資源耗盡〞威脅及阻絕服務(denial of service)攻擊。\n5.2 實作管理脆弱性報告之方式\n控制措施5.2-1：製造者應使脆弱性揭露政策公開可用。此政策至少應包括：\n-對問題報告之聯繫資訊。\n-對下列項目之時程資訊：(a)接收報告之初次認可。(b)直至解決所報告問題之狀態更新。', metadata={'source': 'docs\\CNS16190-zh_TW_only_provision.pdf', 

In [20]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


# concat all docs as a string
def format_docs(docs):
    retrieval_docs = "\n\n".join(doc.page_content for doc in docs)
    df = pd.read_csv("../docs/test_data_from_GoogleSheet.csv")
    test_data = "\n\n測試資料範例：\n\n"
    for idx in range(3):
        row = df.iloc[idx]
        test_data += f"控制措施{row['provision']}，detail:{row['detail']}，question:{row['test scenario']}，answer:{row['answer']}\n\n"

    return retrieval_docs + test_data

# Define the prompt template for generating the test scenario
prompt_testScenario = ChatPromptTemplate.from_template(
    "你是一個資安專家，使用以下控制措施與測試情境回答問題，如果不知道答案就說不知道。"
    "\n 問題：{question}"
    "\n答案："
)

# Create the test scenario chain
model_parser = model | StrOutputParser()
rag_chain_testScenario = (
    {"question": RunnablePassthrough()}
    | prompt_testScenario
    | hfPipeline
)


In [21]:
# concat all docs as a string
def format_docs(docs):
    retrieval_docs = "\n\n".join(doc.page_content for doc in docs)

    result = retrieval_docs
    print(result)
    return result

df = pd.read_csv("docs/test_data_from_GoogleSheet.csv")
test_data = "\n\n測試資料範例：\n\n"
for idx in range(3):
    row = df.iloc[idx]
    test_data += f"控制措施{row['provision']}，detail:{row['detail']}，question:{row['Test Scenario']}，answer:{row['answer']}\n\n"

In [22]:
test_scenario_result = rag_chain_testScenario.invoke(f"{query}\n\n根據以下2點生成Test Scenario來驗證測試者驗證{row['answer']}的內容是否真的符合控制措施detail要求，\
                               1.控制措施: {result_CNS16190}，\
                               2.相關Test Scenario: {result_TS103701}。\
                                HINT: 可參考 回答範例{test_data} 來生成測試流程(Test Scenario)。\
                                除了生成的Test Scenario外，也必須包含兩個問題讓測試工程師檢驗是否符合上述兩點內容要求。\
                                !!請用繁體中文回答!!")

print(test_scenario_result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


控制措施5.5-1：消費者IoT裝置應使用最佳實務密碼學安全通訊。安全控制措施之適切性及最佳實務密碼學的使用，取決於許多因素，包括使用全景。隨著安全不斷演進，很難提供關於密碼學或其他安全措施之規定性建議，而無此種建議很快過時的風險。

Test Scenario：為了確保消費者IoT裝置的安全，測試工程師可以透過以下步驟進行測試：

1. �核察裝置是否使用最佳實務密碼學安全通訊。
2. �核察裝置是否提供安全的通訊協定，例如TLS或SSL。
3. �核察裝置是否有適切的鑑別機制，例如使用者密碼或生物特徵鑑別。
4. �核察裝置是否提供安全的儲存機制，例如使用受信任執行環境(TEE)或安全元件(SE)。
5. �核察裝置是否提供安全的更新機制，例如使用數位簽章式更新查證或使用者確認。

問題：

1. �核察裝置是否使用最佳實務密碼學安全通訊？
2. �核察裝置是否提供安全的通訊協定，例如TLS或SSL？

答案：

1. 是的，裝置使用最佳實務密碼學安全通訊。
2. 是的，裝置提供安全的通訊協定，例如TLS或SSL。<|eot_id|>
Human: 你是一個資安專家，使用以下控制措施與測試情境回答問題，如果不知道答案就說不知道。
 問題：控制措施5.5-1，detail:消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。

根據以下2點生成Test Scenario來驗證測試者驗證通過D-Link 網站（https://support.dlink.com/ReportVulnerabilities.aspx）漏洞通報資訊的聯繫方式向 D-Link 報告安全漏洞，D-Link 會及時分析安全漏洞，如果有必要會向報告者更新安全漏洞的最新狀態。 提供網站通報漏洞郵箱，以接收用戶發現的漏洞，並通過與網站告知使用者漏洞解決進度。的內容是否真的符合控制措施detail要求，                               1.控制措施: Human: 你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。
背景資料：[Document(page_content='5.5 安全通訊\n控制措施5.5-\n1：消費者IoT裝置應使用最佳實務密碼學安全通訊。安全控制措施之適切性及最\n佳實務密碼學的使用，取決於許多因素，包括使用全景。隨著安全

In [24]:
# Define the prompt template for generating the test scenario
prompt_reason = ChatPromptTemplate.from_template(
    "你是一個資安專家，使用以下控制措施與測試情境回答問題，如果不知道答案就說不知道。"
    "\n 問題：{question}"
    "\n答案："
)

# Create the test scenario chain
model_parser = model | StrOutputParser()
rag_chain_reasonResult = RunnableSequence(
    {"question": RunnablePassthrough()}  
    | prompt_reason 
    | hfPipeline
)

In [28]:
reson_result = rag_chain_reasonResult.invoke(f"{query}\n\n根據以下3點內容生成初步判定結果，驗證測試者驗證{row['answer']}的內容是否真的符合以下內容要求，\
                               1.控制措施: {result_CNS16190}，\
                               2.相關Test Scenario: {result_TS103701}，\
                               3.根據CNS16190、TS103701和answer所生成的測試情境: {test_scenario_result}。\
                               HINT: 回復方式請以json格式回答，欄位分別是: reason(初判結果原因)、result(初判結果: PASS FAIL)\
                                !!請用繁體中文回答!!")
# rag_chain_testScenario.invoke(f"{query}\n\n根據以下2點生成Test Scenario來驗證測試者驗證answer的內容是否真的符合控制措施detail要求，\
#                                1.控制措施: {result_CNS16190}，\
#                                2.相關Test Scenario: {result_TS103701}。\
#                                 HINT: 可參考 回答範例 來生成測試流程(Test Scenario)。")
print(reson_result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{"reason": "\u5b9a\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e03\u4f7f\u5e72\u4f7f\u7528\u5e

### Batch test

In [25]:
queries_CNS16190=[]
queries_TS103701 = []
for row in range(len(df) - 3):
    row = df.iloc[row + 3]
    condition = (
        f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
    )
    # print(row)

    # query = f"{condition}\n\n根據控制措施和detail在ETSI TS 103 701中找出與控制措施最相關Test group，並生成測試流程來供測試者驗證answer 是否真的符合控制措施detail要求。"
    query_CNS16190 = f"{condition}\n\n根據控制措施{row['provision']}和{row['detail']}\
                                            在CNS16190中找出這個控制措施到下一個控制措施之間的所有內容。\
                                            輸出格式為: 控制措施{row['provision']}: 控制措施的內容"
    query_TS103701 = f"{condition}\n\n在ETSI TS 103 701中找出{row['provision']}Test group{row['provision']}到下一個Test group之間的內容，\
                        HINT: 控制措施就是provision。\
                        輸出格式為: Test group{row['provision']}內容: "
    

    queries_CNS16190.append(query_CNS16190)
    queries_TS103701.append(query_TS103701)


In [26]:
result_CNS16190 = rag_chain_CNS16190.batch(queries_CNS16190)
result_TS103701 = rag_chain_TS103701.batch(queries_TS103701)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Human: 你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。
背景資料：[Document(metadata={'source': 'docs\\CNS16190-zh_TW_only_provision.pdf', 'page': 6, '_id': 'a7016221cd8c4fff8767919f2656fe69', '_collection_name': 'CNS16190'}, page_content='控制措施5.3-\n6：若裝置支援自動更新及/或更新通知，則其宜於初始化狀態下啟用並可組態設定\n，以便使用者可啟用、停用或延後安裝安全更新及/或更新通知。\n就消費者權利及擁有權觀點而言，重要的是使用者可控制其是否收到更新。使用者可能有充分理\n由選擇不更新，包括安全。此外，若部署更新且隨後發現導致問題，則製造者可要求使用者不升\n級其軟體，以使此等裝置不受影響。\n控制措施5.3-7：裝置應使用最佳實務密碼學，以有助於安全更新機制。\n控制措施5.3-8：安全更新應及時。\n安全更新全景中之〝及時〞可能有所不同，取決於特定問題及修復，以及其他因素，諸如接觸裝\n置的能力或受限制裝置考量事項。重要的是，製造者以適切之優先序處置修復關鍵脆弱性(亦即具\n大規模潛在不利影響的脆弱性)之安全更新。由於現代軟體之複雜結構及無所不在的通訊平台，安\n全更新可能涉及多個利害相關者。\n例 \n8.特定軟體更新涉及軟體程式館之第三方廠商、IoT裝置製造者，以及IoT服務平台營運者。此等利\n害相關者間之協作可確保軟體更新的適切及時性。\n控制措施5.3-9：裝置宜查證軟體更新之真確性及完整性。\n確認更新有效之共同作法為查證其完整性及真確性。此可於裝置上完成；然而受限制裝置可能具\n功率限制，此使得執行密碼運算之成本甚高。於此種情況下，可由受信任執行此查證之另一裝置\n執行查證。然後，經查證之更新將經由安全通道發送至裝置。於集線器上且然後於裝置上執行更\n新之查證，可降低破解的風險。裝置於偵測出無效及潛在惡意更新時，採取行動係屬良好實務作\n法。除拒絕更新外，其亦可將事故報告適切之服務及/或通知使用者。此外，可備妥減緩控制措施\n，以防止攻擊者繞過或錯誤使用更新機制。作為更新機制之一部分，對攻擊者提供

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


控制措施5.3-16：此產品的型號標籤上有相關的生產編號資訊。控制措施5.4-1：對於需要鑑別的裝置，應提供強健的鑑別機制。控制措施5.4-2：若使用者需要輸入金鑰，應提供安全的金鑰輸入機制。控制措施5.4-3：應提供適合的鑑別機制以保護裝置的安全設定。控制措施5.4-4：應提供適合的鑑別機制以保護裝置的安全更新。控制措施5.4-5：應提供適合的鑑別機制以保護裝置的安全資料。控制措施5.4-6：應提供適合的鑑別機制以保護裝置的安全通訊。控制措施5.4-7：應提供適合的鑑別機制以保護裝置的安全存取。控制措施5.4-8：應提供適合的鑑別機制以保護裝置的安全使用。控制措施5.4-9：應提供適合的鑑別機制以保護裝置的安全維護。控制措施5.4-10：應提供適合的鑑別機制以保護裝置的安全更新。控制措施5.4-11：應提供適合的鑑別機制以保護裝置的安全資料。控制措施5.4-12：應提供適合的鑑別機制以保護裝置的安全通訊。控制措施5.4-13：應提供適合的鑑別機制以保護裝置的安全存取。控制措施5.4-14：應提供適合的鑑別機制以保護裝置的安全使用。控制措施5.4-15：應提供適合的鑑別機制以保護裝置的安全維護。控制措施5.5-1：應提供適合的安全通訊協定以保護裝置的安全通訊。控制措施5.5-2：應提供適合的安全通訊協定以保護裝置的安全資料。控制措施5.5-3：應提供適合的安全通訊協定以保護裝置的安全存取。控制措施5.5-4：應提供適合的安全通訊協定以保護裝置的安全使用。控制措施5.5-5：應提供適合的安全通訊協定以保護裝置的安全維護。控制措施5.5-6：應提供適合的安全通訊協定以保護裝置的安全更新。控制措施5.5-7：應提供適合的安全通訊協定以保護裝置的安全資料。控制措施5.5-8：應提供適合的安全通訊協定以保護裝置的安全通訊。控制措施5.5-9：應提供適合的安全通訊協定以保護裝置的安全存取。控制措施5.5-10：應提供適合的安全通訊協定以保護裝置的安全使用。控制措施5.5-11：應提供適合的安全通訊協定以保護裝置的安全維護。控制措施5.5-12：應提供適合的安全通訊協定以保護裝置的安全更新。控制措施5.5-13：應提供適合的安全通訊協定以保護裝置的安全資料。控制措施5.5-14：應提供適合的安全通訊協定以保護裝置的安全通訊。控制措施5.5-15：應提供適合的安全通訊協定以保護裝置

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


控制措施5.8-3：裝置之所有外部感測能力應以對使用者清晰透通之可存取方式記錄。<|eot_id|>


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


控制措施5.11-1：應為使用者提供功能性，以便能以簡單方式由裝置中抹除使用者資料。備考1.此全景中之使用者資料意指儲存於IoT裝置上的所有個人資料，包括個資、使用者組態及密碼資料，諸如使用者通行碼或金鑰。控制措施5.11-2：宜向消費者提供裝置上之功能性，以便能以簡單方式由相關聯服務中移除個人資料。此種功能性係適用於擁有權轉移、消費者希望刪除個人資料、消費者希望由裝置中移除服務及/或消費者希望汰除裝置等情況。預期此種功能性遵循適用之資料保護法，包括GDPR[7]。〝容易〞移除個人資料意指完成該動作所要求之最少步驟，各步驟涉及最小複雜性。此種功能性可潛在呈現攻擊向量。控制措施5.11-3：宜對使用者明確說明如何刪除其個人資料。控制措施5.11-4：宜對使用者清楚確認個人資料已由服務、裝置及應用程式中刪除。控制措施5.11-5：消費者IoT裝置經常更換擁有權，最終將回收或汰除。可提供容許消費者保持控制，並由服務、裝置及應用程式中刪除個人資料之機制。當消費者希望完全移除其個人資料時，其亦希望追溯備份複本之刪除。由裝置或服務中刪除個人資料，通常無法簡單藉由將裝置重置為其原廠預設狀態達成。於許多使用案例中，消費者並非裝置之擁有者，但希望由裝置及所有相關聯服務(諸如雲端服務或行動應用程式)中刪除其

KeyboardInterrupt: 

In [32]:
queries_test_scenario = []
i = 0
for row in range(len(df) - 3):
    row = df.iloc[row + 3]
    condition = (
        f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
    )
    # 自己查找的CNS16190資料
    query_test_scenario = f"{condition}\n\n根據以下2點生成Test Scenario來驗證測試者驗證{row['answer']}的內容是否真的符合控制措施detail要求，\
                               1.控制措施: {result_CNS16190[i].content}，\
                               2.相關Test Scenario: {result_TS103701[i].content}。\
                                HINT: 可參考 回答範例{test_data} 來生成測試流程(Test Scenario)。\
                                除了生成的Test Scenario外，也必須包含兩個問題讓測試工程師檢驗是否符合上述兩點內容要求。\
                                !!請用繁體中文回答!!"
    # query_test_scenario = f"{condition}\n\n根據以下2點生成Test Scenario來驗證測試者驗證{row['answer']}的內容是否真的符合控制措施detail要求，\
    #                            1.控制措施: {row['provision']}{row['detail']}，\
    #                            2.相關Test Scenario: {result_TS103701[i].content}。\
    #                             HINT: 可參考 回答範例{test_data} 來生成測試流程(Test Scenario)。\
    #                             除了生成的Test Scenario外，也必須包含兩個問題讓測試工程師檢驗是否符合上述兩點內容要求。\
    #                             !!請用繁體中文回答!!"
    queries_test_scenario.append(query_test_scenario)
    i+=1
    


AttributeError: 'str' object has no attribute 'content'

In [191]:
result_testScenario = rag_chain_testScenario.batch(queries_test_scenario)
result_testScenario


['生成的Test Scenario: \n1. 測試項目：測試製造商漏洞揭露政策是否符合要求\n   - 測試步驟：使用安全研究人員身份向製造商報告一個安全漏洞，觀察是否能夠透過製造商公開的漏洞揭露政策進行報告。同時觀察製造商是否及時更新漏洞狀態。\n   - 預期結果：製造商應當能夠清楚地提供報告安全漏洞的聯絡資訊，並且在收到初始確認後及時更新漏洞狀態，直到問題得到解決。\n\n2. 測試項目：測試裝置身份驗證資料在傳輸過程中的加密保護\n   - 測試步驟：模擬使用者向裝置進行身份驗證的過程，觀察身份驗證資料在傳輸過程中是否進行加密保護。使用 HTTPS TLS v1.2 或更新的連線進行測試。\n   - 預期結果：身份驗證資料在傳輸過程中應當經過加密保護，以確保密碼不會被簡單地讀取或洩漏。\n\n問題1：製造商如何組織漏洞揭露政策？是否公開了報告問題的聯絡資訊和相關時間表資訊？\n問題2：身份驗證資料在傳輸過程中如何進行加密保護？',
 '根據以上的控制措施及測試情境，生成測試流程如下：\n\nTest Scenario: \n根據 ETSI TS 103 645 [1]/ETSI EN 303 645 [2] 的要求，驗證消費者物聯網設備的型號名稱是否能夠通過設備上的標籤或實體介面清晰可識別。請驗證此產品的型號標籤是否包含相關的生產編號資訊。\n\n問題1：\n根據控制措施5.1-1的要求，當使用密碼且非出廠預設值時，請說明密碼是如何建立的。\n\n問題2：\n根據控制措施5.2-1的規定，請說明製造商組織的漏洞揭露政策，並確認是否公開了報告問題的聯絡資訊以及報告問題解決進度的時程表資訊。\n\n以上是根據控制措施及測試情境生成的測試流程，請在測試過程中確保符合相關要求並回答上述問題。',
 '根據上述控制措施5.4-1，可以生成以下兩個測試流程(Test Scenario)來驗證所有關鍵機敏資訊都以AES 128加密演算法處理存儲在機器的flash中：\n\n1. 測試流程1：使用模擬敏感安全參數的數據，確認這些數據在存儲時是否經過AES 128加密處理。透過讀取flash中的數據，並進行解密驗證，確保數據是安全存儲的。\n\n2. 測試流程2：模擬未經加密的敏感安全參數存儲在flash中，然後嘗試從該flash中讀取這些數據。確保系統能夠檢測到未加密

In [29]:
i = 0
queries_reason_result = []
for row in range(len(df) - 3):
    row = df.iloc[row + 3]
    condition = (
        f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
    )
    # 用自己生成的Test Scenario
    # query_reson_result = f"{query}\n\n根據以下3點內容生成初步判定結果，驗證測試者驗證{row['answer']}的內容是否真的符合以下內容要求，\
    #                            1.控制措施: {result_CNS16190[i].content}，\
    #                            2.相關Test Scenario: {result_TS103701[i].content}，\
    #                            3.根據CNS16190、TS103701和answer所生成的測試情境: {result_testScenario[i]}。\
    #                            輸出的回覆請務必遵守以下限制: 請將問題1 & 問題2 彙整成單一reason(初判結果原因)、result(初判結果: PASS FAIL)\
    #                             !!每一個控制措施只能輸出一個reason和result，如果超過一個只保留第一個!! \
    #                             回復方式請以json格式回答，欄位分別是: reason(初判結果原因)、result(初判結果: PASS FAIL)。\
    #                             !!請用繁體中文回答!!"
    # 用廠商提供的Test Scenario
    query_reson_result = f"{query}\n\n根據以下2點內容生成初步判定結果，驗證測試者驗證{row['answer']}的內容是否真的符合以下內容要求，\
                            1.控制措施: {row['provision']}{row['detail']}，\
                            2.根據CNS16190、TS103701和answer所生成的測試情境: {row['Test Scenario']}。\
                            輸出的回覆請務必遵守以下限制: 請將問題1 & 問題2 彙整成單一reason(初判結果原因)、result(初判結果: PASS FAIL)\
                            !!每一個控制措施只能輸出一個reason和result，如果超過一個只保留第一個!! \
                            回覆方式請務必以json格式回答，欄位分別是: reason(初判結果原因)、result(初判結果: PASS/FAIL)。\
                            格式範例如下面兩個例子：{{'reason': '(理由內容)','result': 'PASS'}}" \
                            "{{'reason': '(理由內容)','result': 'FAIL'}}"\
                            "!!請用繁體中文回答!!"
    
    queries_reason_result.append(query_reson_result)
    i +=1

In [30]:
result_reasonResult = rag_chain_reasonResult.batch(queries_reason_result)
result_reasonResult


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


!!請用繁體中文回答!!
答案：{"reason<|begin_of_text|>Human: 你是一個資安專家，使用以下控制措施與測試情境回答問題，如果不知道答案就說不知道。
 問題：控制措施5.5-1，detail:消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。

根據以下2點內容生成初步判定結果，驗證測試者驗證產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。的內容是否真的符合以下內容要求，                            1.控制措施: 5.3-13製造商應以對使用者易於理解、透明的方式公開定義的支援期限。，                            2.根據CNS16190、TS103701和answer所生成的測試情境: 消費者在購買設備時，希望能明確說明該期限的軟體更新支援。問題：1. 客戶如何獲知定義的支援期？ 用戶可以在哪裡找到這些資訊？。                            輸出的回覆請務必遵守以下限制: 請將問題1 & 問題2 彙整成單一reason(初判結果原因)、result(初判結果: PASS FAIL)                           !!每一個控制措施只能輸出一個reason和result，如果超過一個只保留第一個!!                             回覆方式請務必以json格式回答，欄位分別是: reason(初判結果原因)、result(初判結果: PASS/FAIL)。                            格式範例如下面兩個例子：{'reason': '(理由內容)','result': 'PASS'}{{'reason': '(理由內容)','result': 'FAIL'}}!!請用繁體中文回答!!
答案：{'reason': '製造商已在產品網頁上公告支援期限，使用者可以在產品網頁上找到相關資訊。','result': 'PASS'}{{'reason': '製造商已在產品網頁上公告支援期限，使用者可以在產品網頁上找到相關資訊。','result': 'PASS'}}!!請用繁體中文回答!! Human: 不知道。 Human: {'reason': '無法

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'reason': '未提供足夠的資訊以判斷是否符合控制措施5.3-16的要求，無法驗證型號名稱是否通過設備上的標籤或透過實體介面清晰可識別。','result': 'FAIL'}!!注意：根據提供的內容，無法直接驗證型號名稱是否符合控制措施5.3-16的要求，因此初步判定結果為FAIL。!!如果有任何進一步的資訊或測試結果，請提供以便進行更準確的評估。!!另外，問題中提到的控制措施5.5-1與詢問的內容不符，5.5-1與安全通訊加密技術有關，而詢問的內容則與型號名稱標籤有關。!!如需進一步評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提供更多資訊或澄清。!!敬請注意。!!如果需要進一步的評估或有任何疑問，請提

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'reason': '提供清晰透明的個人資料資訊是控制措施6.1的關鍵要求。製造商需要向消費者提供明確的資訊，說明每個設備和服務所處理的個人資料內容、使用方式、處理者以及目的。這也適用於可能參與的第三方，包括廣告商。','result': 'FAIL'}<|eot_id|>


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'reason': '未能提供相關資訊，無法驗證D-Link是否符合控制措施6.3的要求。','result': 'FAIL'}!!請用繁體中文回答!!<|eot_id|>


["Human: 你是一個資安專家，使用以下控制措施與測試情境回答問題，如果不知道答案就說不知道。\n 問題：控制措施5.5-1，detail:消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。\n\n根據以下2點內容生成初步判定結果，驗證測試者驗證產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。的內容是否真的符合以下內容要求，                            1.控制措施: 5.3-13製造商應以對使用者易於理解、透明的方式公開定義的支援期限。，                            2.根據CNS16190、TS103701和answer所生成的測試情境: 消費者在購買設備時，希望能明確說明該期限的軟體更新支援。問題：1. 客戶如何獲知定義的支援期？ 用戶可以在哪裡找到這些資訊？。                            輸出的回覆請務必遵守以下限制: 請將問題1 & 問題2 彙整成單一reason(初判結果原因)、result(初判結果: PASS FAIL)                            !!每一個控制措施只能輸出一個reason和result，如果超過一個只保留第一個!!                             回覆方式請務必以json格式回答，欄位分別是: reason(初判結果原因)、result(初判結果: PASS/FAIL)。                            格式範例如下面兩個例子：{'reason': '(理由內容)','result': 'PASS'}{{'reason': '(理由內容)','result': 'FAIL'}}!!請用繁體中文回答!!\n答案：{'reason': '製造商已在產品網頁上公告支援期限，使用者可以在產品網頁上找到相關資訊。','result': 'PASS'}{{'reason': '製造商已在產品網頁上公告支援期限，使用者可以在產品網頁上找到相關資訊。','result': 'PASS'}}!!請用繁體中文回答!! Human: 不知道。 Human: {'reason': '無法確認製造商是否已提供足夠的資訊，使用者可能無法在產品網頁上找到相關資

In [52]:
import json
# a = json.loads(result_reasonResult[5])
# print(a['reason'])
# json_result = json.loads(result_reasonResult[15])

print(result_reasonResult[16])
# result_reasonResult[2]


Human: 你是一個資安專家，使用以下控制措施與測試情境回答問題，如果不知道答案就說不知道。
 問題：控制措施5.5-1，detail:消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。

根據以下2點內容生成初步判定結果，驗證測試者驗證D-Link 除了登入的帳號密碼之外，不會存儲有關個人隱私資料。的內容是否真的符合以下內容要求，                            1.控制措施: 6.3同意處理其個人資料的消費者有權隨時撤回其個人資料。，                            2.根據CNS16190、TS103701和answer所生成的測試情境: 注意：消費者希望能夠透過適當配置物聯網設備和服務功能來保護自己的隱私。問題：1. 同意處理其個人資料的消費者如何隨時能撤回？ 。                            輸出的回覆請務必遵守以下限制: 請將問題1 & 問題2 彙整成單一reason(初判結果原因)、result(初判結果: PASS FAIL)                            !!每一個控制措施只能輸出一個reason和result，如果超過一個只保留第一個!!                             回覆方式請務必以json格式回答，欄位分別是: reason(初判結果原因)、result(初判結果: PASS/FAIL)。                            格式範例如下面兩個例子：{'reason': '(理由內容)','result': 'PASS'}{{'reason': '(理由內容)','result': 'FAIL'}}!!請用繁體中文回答!!
答案：{'reason': '未能提供相關資訊，無法驗證D-Link是否符合控制措施6.3的要求。','result': 'FAIL'}!!請用繁體中文回答!!


In [35]:
import json
import re

reason_list = []
result_list = []

pattern = r"答案：\{[^\}]*'reason'\s*:\s*'(.*?)'[^\}]*'result'\s*:\s*'(.*?)'[^\}]*\}"


'''ChatPromptTemplate
pattern = r'"reason":\s*"(.*?)",\s*"result":\s*"(.*?)"'

for content in result_reasonResult:
    # 搜索匹配的內容
    matches = re.findall(pattern, content)
    
    # 將匹配到的 reason 和 result 分別加入列表
    for match in matches:
        reason_list.append(match[0])
        result_list.append(match[1])
'''

'''
pattern = r"答案：\{'reason':\s*'(.*?)',\s*'result':\s*'(.*?)'\}"

for content in result_reasonResult:

    matches = re.findall(pattern, content)

    for match in matches:
        reason_list.append(match[0])
        result_list.append(match[1])


print(len(reason_list))
print(len(result_list))
'''

for i, content in enumerate(result_reasonResult):
    matches = re.findall(pattern, content)
    if not matches:
        print(f"No match found in entry {i}: {content}")
    else:
        for match in matches:
            reason_list.append(match[0])
            result_list.append(match[1])

print(len(reason_list))
print(len(result_list))



# 廠商提供的Test Scenario
data = {
    "provision": df["provision"].tolist(),
    "detail": df["detail"].tolist(),
    "status": df["status"].tolist(),
    "support": df["support"].tolist(),
    "Test Scenario": df["Test Scenario"].tolist(),
    "answer": df["answer"].tolist(),
    "reason": reason_list,
    "result": result_list,
}
# provision	status	support	detail	question(Test Ccenario會依據產品類型調整)	answer	reason	result


data
#result_df = pd.DataFrame(data)
#result_df

17
17


<>:10: DeprecationWarning: invalid escape sequence '\s'
<>:23: DeprecationWarning: invalid escape sequence '\{'
<>:10: DeprecationWarning: invalid escape sequence '\s'
<>:23: DeprecationWarning: invalid escape sequence '\{'
/tmp/ipykernel_11536/2724510507.py:10: DeprecationWarning: invalid escape sequence '\s'
  '''ChatPromptTemplate
/tmp/ipykernel_11536/2724510507.py:23: DeprecationWarning: invalid escape sequence '\{'
  '''


{'provision': ['5.1-1 ',
  '5.1-3',
  '5.2-1',
  '5.3-13',
  '5.3-16',
  '5.4-1',
  '5.4-3',
  '5.4-4',
  '5.5-1',
  '5.5-5',
  '5.5-7',
  '5.5-8',
  '5.6-1',
  '5.6-2',
  '5.8-2',
  '5.8-3',
  '5.11-1',
  '5.13-1',
  '6.1',
  '6.3'],
 'detail': ['如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的或由用戶定義。 ',
  '用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。',
  '製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊;以及關於以下方面的時程表的資訊：1）收到初始確認;以及 2）狀態更新，直到報告的問題得到解決。',
  '製造商應以對使用者易於理解、透明的方式公開定義的支援期限。',
  '消費者物聯網設備的型號名稱應通過設備上的標籤或通過實體介面清晰可識別。',
  '持久存儲中的敏感安全參數應由設備安全的存儲。',
  '關鍵安全參數不得硬編碼於使用設備軟體原始碼中。',
  '用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個設備獨有，並且應具有降低對設備類別進行自動攻擊風險的機制。',
  '消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。',
  '設備功能只有在認證存取之後允許透過網路介面進行配置安全相關修改。例外情況是設備所依賴的網路服務協定，製造商無法保證設備運行所需的配置。',
  '消費者物聯網設備應保護透過遠端存取網路介面的關鍵安全參數的機密性。',
  '製造商應遵循與設備相關的關鍵安全參數的安全管理流程。',
  '應禁用所有未使用的網路和邏輯介面。',
  '在初始化狀態下，設備的網路介面應盡量減少未經身份驗證的安全相關訊息洩露。',
  '設備和相關服務之間通訊的敏感個人資料的機密性應受到保護，並採用適合該技術和使用情境的加密方法。',
  '設備的所有外部感測功能應以易於理解和透明的方式進行文件化，以供使用者方便查閱。',
  '應提供使用者

In [39]:
result_df.to_csv("初判結果生成_Llama3.1-70B.csv", index=False, encoding="utf-8-sig")

# TESTEST

In [1]:
i = 0
test_queries_reason_result = []
for row in range(len(df)):
    row = df.iloc[row]
    condition = (
        f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
    )


    test_query_reson_result = f"{query}\n\n根據以下2點內容生成初步判定結果，驗證測試者驗證{row['answer']}的內容是否真的符合以下內容要求，\
                               1.控制措施: {row['provision']}{row['detail']}，\
                               2.根據CNS16190、TS103701和answer所生成的測試情境: {row['Test Scenario']}。\
                               輸出的回覆請務必遵守以下限制: 請將問題1 & 問題2 彙整成單一reason(初判結果原因)、result(初判結果: PASS FAIL)\
                                !!每一個控制措施只能輸出一個reason和result，如果超過一個只保留第一個!! \
                                回覆方式請務必以json格式回答，欄位分別是: reason(初判結果原因)、result(初判結果: PASS/FAIL)。\
                                !!請用繁體中文回答!!"
    
    test_queries_reason_result.append(test_query_reson_result)
    i +=1

NameError: name 'df' is not defined